# S-C Evidence Pipeline - Stage 3: Post-Processing

This notebook applies post-processing to reranker outputs:
- **No-Evidence Detection:** Identifies queries where the post has no supporting evidence
- **Dynamic-K Selection:** Adaptively selects the number of results based on score distribution
- **Score Calibration:** Converts raw reranker scores to calibrated probabilities

## Prerequisites
- Run `sc_retrieval_pipeline.ipynb` first (Stage 1: Retrieval)
- Run `sc_reranker_pipeline_no_postProcessing.ipynb` second (Stage 2: Reranking)

## Best Post-Processing Config (HPO on Reranker Scores - Trial 107)
- **No-Evidence Detection:** score_gap method (optimized for reranker logit distribution)
  - score_gap_threshold=1.148 (reranker logits range: -3.75 to 0.025)
- **Dynamic-K Selection:** score_gap method, min_k=5, max_k=16, ratio=0.549
- **Calibration:** platt scaling

In [1]:
import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Tuple, Set
from tqdm.auto import tqdm

import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from torch.optim import AdamW

# Add project root to path
project_root = Path("..")
sys.path.insert(0, str(project_root))

from final_sc_review.metrics.ranking import ndcg_at_k, recall_at_k, mrr_at_k, map_at_k
from final_sc_review.reranker.losses import HybridRerankerLoss
from final_sc_review.postprocessing.no_evidence import (
    NoEvidenceDetector,
    compute_no_evidence_metrics,
)
from final_sc_review.postprocessing.dynamic_k import DynamicKSelector
from final_sc_review.postprocessing.calibration import ScoreCalibrator, compute_ece

/home/user/miniforge3/envs/jina-reranker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Configuration

In [2]:
# Best Reranker Config (Trial 33, nDCG@10=0.8205)
BEST_RERANKER_PARAMS = {
    'batch_size': 1,
    'num_epochs': 1,
    'learning_rate': 4.447467238603695e-05,
    'weight_decay': 8.769982161626777e-05,
    'grad_accum': 2,
    'pointwise_type': 'bce',
    'pairwise_type': 'pairwise_softplus',
    'listwise_type': 'lambda',
    'w_list': 1.0755666826190335,
    'w_pair': 1.8398728897689836,
    'w_point': 0.813832693617893,
    'temperature': 0.9342605824607415,
    'sigma': 1.5735217400312576,
    'margin': 0.7247599691970003,
    'max_pairs': 100,
    'lora_r': 16,
    'lora_alpha': 16,
    'lora_dropout': 0.05,
}

# Best Post-Processing Config (HPO on RERANKER SCORES - Trial 107)
# Optimized for reranker logit distribution (range -3.75 to 0.025)
# Combined score: 0.7355 (nDCG=0.9815, NE_F1=0.149)
BEST_POSTPROC_PARAMS = {
    'ne_method': 'score_gap',
    'max_score_threshold': 4.293236027208229,
    'score_std_threshold': 2.3714399938995943,
    'score_gap_threshold': 1.1481314290830258,
    'dk_method': 'score_gap',
    'min_k': 5,
    'max_k': 16,
    'score_gap_ratio': 0.5492568532194798,
    'cal_method': 'platt',
}

# Pipeline config
RERANKER_MODEL_ID = "jinaai/jina-reranker-v3"
TOP_K_RETRIEVER = 20
N_FOLDS = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Input/Output paths
INPUT_DIR = project_root / "outputs" / "retrieval_candidates"
OUTPUT_DIR = project_root / "outputs" / "5fold_results_with_postproc"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Device: {DEVICE}")
print(f"Reranker: {RERANKER_MODEL_ID}")
print(f"Post-processing: NE={BEST_POSTPROC_PARAMS['ne_method']}, DK={BEST_POSTPROC_PARAMS['dk_method']}")

Device: cuda
Reranker: jinaai/jina-reranker-v3
Post-processing: NE=score_gap, DK=score_gap


## 2. Load Retrieval Candidates

In [3]:
# Load configuration metadata
with open(INPUT_DIR / "config_metadata.json", "r") as f:
    config_metadata = json.load(f)

print("Retrieval Configuration:")
for key, value in config_metadata.items():
    print(f"  - {key}: {value}")

Retrieval Configuration:
  - retriever_name: nv-embed-v2
  - top_k_retriever: 20
  - n_folds: 5
  - total_posts: 1477
  - total_queries: 14770


In [4]:
# Load retrieval candidates
with open(INPUT_DIR / "retrieval_candidates.pkl", "rb") as f:
    all_fold_data = pickle.load(f)

print(f"\nLoaded {len(all_fold_data)} folds:")
for fold_name, fold_data in all_fold_data.items():
    print(f"  - {fold_name}: {len(fold_data['train_data'])} train, {len(fold_data['val_data'])} val samples")


Loaded 5 folds:
  - fold_1: 10810 train, 2700 val samples
  - fold_2: 10790 train, 2720 val samples
  - fold_3: 10810 train, 2700 val samples
  - fold_4: 10770 train, 2740 val samples
  - fold_5: 10860 train, 2650 val samples


## 3. Define Training and Post-Processing Functions

In [5]:
def train_reranker(train_data, params, verbose=True):
    """Train a LoRA-adapted reranker on the training data."""
    tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL_ID, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load model with num_labels=1 for regression-style reranking
    model = AutoModelForSequenceClassification.from_pretrained(
        RERANKER_MODEL_ID,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        num_labels=1
    )
    model.config.pad_token_id = tokenizer.pad_token_id
    
    # LoRA target modules for jina-reranker-v3 (Qwen3-based)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    lora_config = LoraConfig(
        r=params['lora_r'],
        lora_alpha=params['lora_alpha'],
        lora_dropout=params['lora_dropout'],
        target_modules=target_modules,
        bias="none",
        task_type="SEQ_CLS",
    )
    model = get_peft_model(model, lora_config)
    model = model.to(DEVICE)
    
    loss_fn = HybridRerankerLoss(
        pointwise_type=params['pointwise_type'],
        pairwise_type=params['pairwise_type'],
        listwise_type=params['listwise_type'],
        w_point=params['w_point'],
        w_pair=params['w_pair'],
        w_list=params['w_list'],
        temperature=params['temperature'],
        sigma=params['sigma'],
        margin=params['margin'],
    )
    
    optimizer = AdamW(model.parameters(), lr=params['learning_rate'], weight_decay=params['weight_decay'])
    
    model.train()
    total_loss = 0
    step = 0
    
    train_data_shuffled = train_data.copy()
    np.random.shuffle(train_data_shuffled)
    
    for epoch in range(params['num_epochs']):
        for query_data in train_data_shuffled:
            query_text = query_data['query']
            candidates = query_data['candidates'][:TOP_K_RETRIEVER]
            
            if len(candidates) < 2:
                continue
            
            texts = [[query_text, c['text']] for c in candidates]
            labels = torch.tensor([c['label'] for c in candidates], dtype=torch.float32, device=DEVICE)
            
            inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            
            outputs = model(**inputs)
            scores = outputs.logits.squeeze(-1)
            
            loss = loss_fn(scores, labels)
            loss = loss / params['grad_accum']
            loss.backward()
            
            step += 1
            if step % params['grad_accum'] == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            total_loss += loss.item() * params['grad_accum']
    
    avg_loss = total_loss / step if step > 0 else 0
    if verbose:
        print(f"  Training done. Steps: {step}, Avg loss: {avg_loss:.4f}")
    
    return model, tokenizer

In [6]:
def fit_calibrator(model, tokenizer, train_data, cal_method='temperature'):
    """Fit score calibrator on training data."""
    model.eval()
    all_scores = []
    all_labels = []
    
    with torch.no_grad():
        for query_data in tqdm(train_data, desc="Fitting calibrator"):
            query_text = query_data['query']
            candidates = query_data['candidates'][:TOP_K_RETRIEVER]
            
            if len(candidates) < 2:
                continue
            
            texts = [[query_text, c['text']] for c in candidates]
            inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            
            outputs = model(**inputs)
            # Convert to float32 before numpy (bfloat16 not supported by numpy)
            scores = outputs.logits.squeeze(-1).float().cpu().numpy()
            labels = [c['label'] for c in candidates]
            
            all_scores.extend(scores)
            all_labels.extend(labels)
    
    calibrator = ScoreCalibrator(method=cal_method)
    calibrator.fit(np.array(all_scores), np.array(all_labels))
    
    return calibrator

In [7]:
def assess_with_postprocessing(model, tokenizer, calibrator, val_data, postproc_params, verbose=True):
    """Assess reranker with post-processing on validation data."""
    model.eval()
    
    # Initialize post-processing modules
    no_evidence_detector = NoEvidenceDetector(
        method=postproc_params['ne_method'],
        max_score_threshold=postproc_params['max_score_threshold'],
        score_std_threshold=postproc_params['score_std_threshold'],
        score_gap_threshold=postproc_params.get('score_gap_threshold', 0.1),
    )
    
    dynamic_k_selector = DynamicKSelector(
        method=postproc_params['dk_method'],
        min_k=postproc_params['min_k'],
        max_k=postproc_params['max_k'],
        score_gap_ratio=postproc_params['score_gap_ratio'],
    )
    
    # Metrics storage
    all_results = {k: [] for k in [1, 5, 10, 20]}
    dynamic_k_results = []
    detailed_results = []
    
    # No-evidence detection tracking
    ne_predictions = []
    ne_groundtruth = []
    
    with torch.no_grad():
        for query_data in tqdm(val_data, desc="Assessing", disable=not verbose):
            query_text = query_data['query']
            candidates = query_data['candidates'][:TOP_K_RETRIEVER]
            gold_uids = set(query_data['gold_uids'])
            is_no_evidence = query_data.get('is_no_evidence', len(gold_uids) == 0)
            
            if len(candidates) < 2:
                continue
            
            # Score candidates
            texts = [[query_text, c['text']] for c in candidates]
            inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
            
            outputs = model(**inputs)
            # Convert to float32 before numpy (bfloat16 not supported by numpy)
            scores = outputs.logits.squeeze(-1).float().cpu().numpy()
            
            # Rerank by score
            ranked_indices = np.argsort(scores)[::-1]
            reranked_candidates = [candidates[i] for i in ranked_indices]
            reranked_ids = [candidates[i]['sent_uid'] for i in ranked_indices]
            reranked_scores = [float(scores[i]) for i in ranked_indices]
            
            # Calibrate scores
            try:
                calibrated_probs = calibrator.calibrate(np.array(reranked_scores)).tolist()
            except Exception:
                calibrated_probs = reranked_scores
            
            # No-evidence detection
            ne_result = no_evidence_detector.detect(reranked_scores, calibrated_probs)
            ne_predictions.append(ne_result.has_evidence)
            ne_groundtruth.append(not is_no_evidence)
            
            # Dynamic-k selection
            dk_result = dynamic_k_selector.select_k(reranked_scores, calibrated_probs)
            dynamic_k = dk_result.selected_k
            
            # Final output based on post-processing
            if ne_result.has_evidence:
                final_ids = reranked_ids[:dynamic_k]
                final_scores = reranked_scores[:dynamic_k]
                final_probs = calibrated_probs[:dynamic_k]
            else:
                final_ids = []
                final_scores = []
                final_probs = []
            
            # Compute metrics (only for queries with gold evidence)
            if gold_uids:
                retriever_ranking = [c['sent_uid'] for c in candidates]
                
                for k in [1, 5, 10, 20]:
                    ret_ndcg = ndcg_at_k(gold_uids, retriever_ranking, k)
                    ret_recall = recall_at_k(gold_uids, retriever_ranking, k)
                    rerank_ndcg = ndcg_at_k(gold_uids, reranked_ids, k)
                    rerank_recall = recall_at_k(gold_uids, reranked_ids, k)
                    final_ndcg = ndcg_at_k(gold_uids, final_ids, k) if final_ids else 0.0
                    final_recall = recall_at_k(gold_uids, final_ids, k) if final_ids else 0.0
                    
                    all_results[k].append({
                        'ret_ndcg': ret_ndcg, 'ret_recall': ret_recall,
                        'rerank_ndcg': rerank_ndcg, 'rerank_recall': rerank_recall,
                        'final_ndcg': final_ndcg, 'final_recall': final_recall,
                    })
                
                dk_ndcg = ndcg_at_k(gold_uids, reranked_ids, dynamic_k)
                dk_recall = recall_at_k(gold_uids, reranked_ids, dynamic_k)
                dynamic_k_results.append({'dynamic_k': dynamic_k, 'dk_ndcg': dk_ndcg, 'dk_recall': dk_recall})
            
            detailed_results.append({
                'post_id': query_data['post_id'],
                'criterion_id': query_data['criterion_id'],
                'query_text': query_text,
                'gold_uids': list(gold_uids),
                'is_no_evidence_gt': is_no_evidence,
                'ne_detected': not ne_result.has_evidence,
                'ne_confidence': ne_result.confidence,
                'dynamic_k': dynamic_k,
                'n_final_results': len(final_ids),
                'final_ids': final_ids,
                'final_scores': final_scores,
                'final_probs': final_probs,
                'reranked_ids': reranked_ids,
                'reranked_scores': reranked_scores,
            })
    
    # Aggregate metrics
    metrics = {}
    for k in [1, 5, 10, 20]:
        if all_results[k]:
            metrics[f'ret_ndcg@{k}'] = np.mean([r['ret_ndcg'] for r in all_results[k]])
            metrics[f'ret_recall@{k}'] = np.mean([r['ret_recall'] for r in all_results[k]])
            metrics[f'rerank_ndcg@{k}'] = np.mean([r['rerank_ndcg'] for r in all_results[k]])
            metrics[f'rerank_recall@{k}'] = np.mean([r['rerank_recall'] for r in all_results[k]])
            metrics[f'final_ndcg@{k}'] = np.mean([r['final_ndcg'] for r in all_results[k]])
            metrics[f'final_recall@{k}'] = np.mean([r['final_recall'] for r in all_results[k]])
    
    if dynamic_k_results:
        metrics['avg_dynamic_k'] = np.mean([r['dynamic_k'] for r in dynamic_k_results])
        metrics['dk_ndcg'] = np.mean([r['dk_ndcg'] for r in dynamic_k_results])
        metrics['dk_recall'] = np.mean([r['dk_recall'] for r in dynamic_k_results])
    
    ne_metrics = compute_no_evidence_metrics(ne_predictions, ne_groundtruth)
    metrics['ne_f1'] = ne_metrics['f1']
    metrics['ne_precision'] = ne_metrics['precision']
    metrics['ne_recall'] = ne_metrics['recall']
    metrics['ne_accuracy'] = ne_metrics['accuracy']
    metrics['n_queries'] = len(all_results[10])
    metrics['n_total'] = len(detailed_results)
    
    return metrics, detailed_results

## 4. Run 5-Fold Cross-Validation with Post-Processing

In [8]:
# Store results
fold_metrics = []
all_detailed_results = []

for fold_idx in range(N_FOLDS):
    fold_name = f'fold_{fold_idx + 1}'
    fold_data = all_fold_data[fold_name]
    
    print(f"\n{'='*60}")
    print(f"FOLD {fold_idx + 1}/{N_FOLDS}")
    print(f"{'='*60}")
    
    train_data = fold_data['train_data']
    val_data = fold_data['val_data']
    
    print(f"Training samples: {len(train_data)}, Validation samples: {len(val_data)}")
    
    # Train reranker
    print("\nTraining reranker...")
    model, tokenizer = train_reranker(train_data, BEST_RERANKER_PARAMS, verbose=True)
    
    # Fit calibrator on training data
    print("\nFitting calibrator...")
    calibrator = fit_calibrator(model, tokenizer, train_data, cal_method=BEST_POSTPROC_PARAMS['cal_method'])
    
    # Assess with post-processing
    print("\nAssessing with post-processing...")
    metrics, detailed = assess_with_postprocessing(
        model, tokenizer, calibrator, val_data, BEST_POSTPROC_PARAMS, verbose=True
    )
    
    # Add fold info to detailed results
    for d in detailed:
        d['fold'] = fold_idx + 1
    
    fold_metrics.append(metrics)
    all_detailed_results.extend(detailed)
    
    # Print fold results
    print(f"\nFold {fold_idx + 1} Results:")
    print(f"  Queries with evidence: {metrics['n_queries']}")
    print(f"  Retriever nDCG@10: {metrics['ret_ndcg@10']:.4f}")
    print(f"  Reranker nDCG@10:  {metrics['rerank_ndcg@10']:.4f}")
    print(f"  Final nDCG@10:     {metrics['final_ndcg@10']:.4f}")
    print(f"  Dynamic-K nDCG:    {metrics['dk_ndcg']:.4f} (avg k={metrics['avg_dynamic_k']:.1f})")
    print(f"  No-Evidence F1:    {metrics['ne_f1']:.4f}")
    
    # Clean up
    del model, tokenizer, calibrator
    torch.cuda.empty_cache()


FOLD 1/5
Training samples: 10810, Validation samples: 2700

Training reranker...


Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at jinaai/jina-reranker-v3 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  Training done. Steps: 10810, Avg loss: 0.1259

Fitting calibrator...


Fitting calibrator:   0%|          | 0/10810 [00:00<?, ?it/s]

Fitting calibrator:   0%|          | 5/10810 [00:00<04:06, 43.88it/s]

...

Assessing: 100%|█████████▉| 2637/2650 [01:00<00:00, 43.36it/s]

Assessing: 100%|█████████▉| 2642/2650 [01:00<00:00, 42.55it/s]

Assessing: 100%|█████████▉| 2647/2650 [01:00<00:00, 42.07it/s]

Assessing: 100%|██████████| 2650/2650 [01:00<00:00, 43.70it/s]


Fold 5 Results:
  Queries with evidence: 241
  Retriever nDCG@10: 0.7192
  Reranker nDCG@10:  0.7549
  Final nDCG@10:     0.4646
  Dynamic-K nDCG:    0.7283 (avg k=5.0)
  No-Evidence F1:    0.3504


## 5. Aggregate Results Across Folds

In [9]:
# Compute mean and std across folds
aggregated = {}
for key in fold_metrics[0].keys():
    if key in ['n_queries', 'n_total']:
        aggregated[key] = sum(m[key] for m in fold_metrics)
    else:
        values = [m[key] for m in fold_metrics]
        aggregated[f'{key}_mean'] = np.mean(values)
        aggregated[f'{key}_std'] = np.std(values)

# Print comprehensive results table
print("\n" + "="*90)
print("5-FOLD CROSS-VALIDATION RESULTS WITH POST-PROCESSING")
print(f"Model: NV-Embed-v2 + {RERANKER_MODEL_ID.split('/')[-1]} + Post-Processing")
print("="*90)

print(f"\nTotal queries assessed: {aggregated['n_queries']} (with evidence), {aggregated['n_total']} (total)")

print(f"\n{'Metric':<18} {'@1':>14} {'@5':>14} {'@10':>14} {'@20':>14}")
print("-"*75)

for metric_name in ['ndcg', 'recall']:
    row = f"Ret {metric_name.upper():<12}"
    for k in [1, 5, 10, 20]:
        mean_val = aggregated.get(f'ret_{metric_name}@{k}_mean', 0)
        std_val = aggregated.get(f'ret_{metric_name}@{k}_std', 0)
        row += f" {mean_val:.4f}+/-{std_val:.3f}"
    print(row)

print("-"*75)

for metric_name in ['ndcg', 'recall']:
    row = f"Rerank {metric_name.upper():<10}"
    for k in [1, 5, 10, 20]:
        mean_val = aggregated.get(f'rerank_{metric_name}@{k}_mean', 0)
        std_val = aggregated.get(f'rerank_{metric_name}@{k}_std', 0)
        row += f" {mean_val:.4f}+/-{std_val:.3f}"
    print(row)

print("-"*75)

for metric_name in ['ndcg', 'recall']:
    row = f"Final {metric_name.upper():<11}"
    for k in [1, 5, 10, 20]:
        mean_val = aggregated.get(f'final_{metric_name}@{k}_mean', 0)
        std_val = aggregated.get(f'final_{metric_name}@{k}_std', 0)
        row += f" {mean_val:.4f}+/-{std_val:.3f}"
    print(row)

print("-"*75)


5-FOLD CROSS-VALIDATION RESULTS WITH POST-PROCESSING
Model: NV-Embed-v2 + jina-reranker-v3 + Post-Processing

Total queries assessed: 1276 (with evidence), 13510 (total)

Metric                         @1             @5            @10            @20
---------------------------------------------------------------------------
Ret NDCG         0.5222+/-0.032 0.6689+/-0.025 0.6955+/-0.019 0.7128+/-0.017
Ret RECALL       0.4849+/-0.023 0.7952+/-0.032 0.8739+/-0.018 0.9393+/-0.018
---------------------------------------------------------------------------
Rerank NDCG       0.6018+/-0.038 0.7251+/-0.029 0.7473+/-0.023 0.7587+/-0.022
Rerank RECALL     0.5651+/-0.031 0.8314+/-0.030 0.8963+/-0.014 0.9393+/-0.018
---------------------------------------------------------------------------
Final NDCG        0.4212+/-0.015 0.4639+/-0.016 0.4639+/-0.016 0.4639+/-0.016
Final RECALL      0.4098+/-0.015 0.5043+/-0.022 0.5043+/-0.022 0.5043+/-0.022
-------------------------------------------------------

In [10]:
# Dynamic-K and No-Evidence metrics
print("\nDynamic-K Selection Metrics:")
print(f"  Average Dynamic K:  {aggregated['avg_dynamic_k_mean']:.2f} +/- {aggregated['avg_dynamic_k_std']:.2f}")
print(f"  Dynamic-K nDCG:     {aggregated['dk_ndcg_mean']:.4f} +/- {aggregated['dk_ndcg_std']:.4f}")
print(f"  Dynamic-K Recall:   {aggregated['dk_recall_mean']:.4f} +/- {aggregated['dk_recall_std']:.4f}")

print("\nNo-Evidence Detection Metrics:")
print(f"  F1 Score:           {aggregated['ne_f1_mean']:.4f} +/- {aggregated['ne_f1_std']:.4f}")
print(f"  Precision:          {aggregated['ne_precision_mean']:.4f} +/- {aggregated['ne_precision_std']:.4f}")
print(f"  Recall:             {aggregated['ne_recall_mean']:.4f} +/- {aggregated['ne_recall_std']:.4f}")
print(f"  Accuracy:           {aggregated['ne_accuracy_mean']:.4f} +/- {aggregated['ne_accuracy_std']:.4f}")


Dynamic-K Selection Metrics:
  Average Dynamic K:  5.00 +/- 0.00
  Dynamic-K nDCG:     0.7251 +/- 0.0289
  Dynamic-K Recall:   0.8314 +/- 0.0299

No-Evidence Detection Metrics:
  F1 Score:           0.3432 +/- 0.0438
  Precision:          0.2469 +/- 0.0421
  Recall:             0.5741 +/- 0.0495
  Accuracy:           0.7881 +/- 0.0432


In [11]:
# Improvement comparison
print("\nImprovement Summary (nDCG@10):")
ret = aggregated['ret_ndcg@10_mean']
rerank = aggregated['rerank_ndcg@10_mean']
final = aggregated['final_ndcg@10_mean']
dk = aggregated['dk_ndcg_mean']

print(f"  Retriever -> Reranker: {ret:.4f} -> {rerank:.4f} (+{rerank-ret:.4f}, +{(rerank-ret)/ret*100:.1f}%)")
print(f"  Reranker -> Final:     {rerank:.4f} -> {final:.4f} ({final-rerank:+.4f}, {(final-rerank)/rerank*100:+.1f}%)")
print(f"  Dynamic-K nDCG:        {dk:.4f}")


Improvement Summary (nDCG@10):
  Retriever -> Reranker: 0.6955 -> 0.7473 (+0.0518, +7.4%)
  Reranker -> Final:     0.7473 -> 0.4639 (-0.2834, -37.9%)
  Dynamic-K nDCG:        0.7251


In [12]:
# Create DataFrame for fold-by-fold comparison
fold_df = pd.DataFrame(fold_metrics)
fold_df.index = [f"Fold {i+1}" for i in range(len(fold_metrics))]

# Show key metrics per fold
key_metrics = ['ret_ndcg@10', 'rerank_ndcg@10', 'final_ndcg@10', 'dk_ndcg', 'ne_f1', 'avg_dynamic_k', 'n_queries']
print("\nPer-Fold Metrics:")
display(fold_df[key_metrics].round(4))


Per-Fold Metrics:


,ret_ndcg@10,rerank_ndcg@10,final_ndcg@10,dk_ndcg,ne_f1,avg_dynamic_k,n_queries
Fold 1,0.6954,0.7588,0.4792,0.7425,0.2974,5.0,255
Fold 2,0.7115,0.7657,0.4331,0.7525,0.3103,5.0,245
Fold 3,0.6870,0.7558,0.4768,0.7324,0.4225,5.0,250
Fold 4,0.6644,0.7012,0.4659,0.6697,0.3352,5.0,285
Fold 5,0.7192,0.7549,0.4646,0.7283,0.3504,5.0,241


## 6. Detailed Analysis

In [13]:
# Convert to DataFrame for analysis
detailed_df = pd.DataFrame([
    {
        'fold': d['fold'],
        'post_id': d['post_id'],
        'criterion_id': d['criterion_id'],
        'query_text': d['query_text'][:50] + '...',
        'n_gold': len(d['gold_uids']),
        'is_no_evidence_gt': d['is_no_evidence_gt'],
        'ne_detected': d['ne_detected'],
        'dynamic_k': d['dynamic_k'],
        'n_final_results': d['n_final_results'],
    }
    for d in all_detailed_results
])

print(f"Total detailed results: {len(detailed_df)}")
display(detailed_df.head(10))

Total detailed results: 13510


,fold,post_id,criterion_id,query_text,n_gold,is_no_evidence_gt,ne_detected,dynamic_k,n_final_results
0,1,s_993_557,A.1,"Depressed mood most of the day, nearly every d...",0,True,True,5,0
1,1,s_993_557,A.2,Markedly diminished interest or pleasure in al...,0,True,True,5,0
2,1,s_993_557,A.3,Significant weight loss when not dieting or we...,0,True,True,5,0
3,1,s_993_557,A.4,Insomnia or hypersomnia nearly every day....,0,True,True,5,0
4,1,s_993_557,A.5,Psychomotor agitation or retardation nearly ev...,0,True,True,5,0
5,1,s_993_557,A.6,Fatigue or loss of energy nearly every day....,0,True,True,5,0
6,1,s_993_557,A.7,Feelings of worthlessness or excessive or inap...,1,False,False,5,4
7,1,s_993_557,A.8,"Diminished ability to think or concentrate, or...",0,True,True,5,0
8,1,s_993_557,A.9,Recurrent thoughts of death (not just fear of ...,0,True,True,5,0
9,1,s_993_557,A.10,Special case criterion for additional clinical...,0,True,True,5,0


In [14]:
# No-evidence detection analysis
print("\nNo-Evidence Detection Analysis:")
print(f"  Total queries: {len(detailed_df)}")
print(f"  Queries with no evidence (GT): {detailed_df['is_no_evidence_gt'].sum()}")
print(f"  Queries detected as no evidence: {detailed_df['ne_detected'].sum()}")

# Confusion matrix
tp = ((~detailed_df['is_no_evidence_gt']) & (~detailed_df['ne_detected'])).sum()
tn = (detailed_df['is_no_evidence_gt'] & detailed_df['ne_detected']).sum()
fp = (detailed_df['is_no_evidence_gt'] & (~detailed_df['ne_detected'])).sum()
fn = ((~detailed_df['is_no_evidence_gt']) & detailed_df['ne_detected']).sum()

print(f"\n  Confusion Matrix:")
print(f"    TP (has evidence, correct): {tp}")
print(f"    TN (no evidence, correct):  {tn}")
print(f"    FP (no evidence, missed):   {fp}")
print(f"    FN (has evidence, missed):  {fn}")


No-Evidence Detection Analysis:
  Total queries: 13510
  Queries with no evidence (GT): 12234
  Queries detected as no evidence: 10450

  Confusion Matrix:
    TP (has evidence, correct): 735
    TN (no evidence, correct):  9909
    FP (no evidence, missed):   2325
    FN (has evidence, missed):  541


In [15]:
# Dynamic-K distribution
print("\nDynamic-K Distribution:")
dk_dist = detailed_df['dynamic_k'].value_counts().sort_index()
print(dk_dist)

print(f"\nDynamic-K Statistics:")
print(f"  Mean:   {detailed_df['dynamic_k'].mean():.2f}")
print(f"  Median: {detailed_df['dynamic_k'].median():.1f}")
print(f"  Std:    {detailed_df['dynamic_k'].std():.2f}")
print(f"  Min:    {detailed_df['dynamic_k'].min()}")
print(f"  Max:    {detailed_df['dynamic_k'].max()}")


Dynamic-K Distribution:
dynamic_k
5    13510
Name: count, dtype: int64

Dynamic-K Statistics:
  Mean:   5.00
  Median: 5.0
  Std:    0.00
  Min:    5
  Max:    5


## 6.1 Comprehensive Binary Classification Metrics (No-Evidence Detection)

This section evaluates the binary classification performance for detecting whether a query has evidence or not.

In [16]:
# Comprehensive Binary Classification Metrics for No-Evidence Detection
# Ground truth: has_evidence (True = has evidence, False = no evidence)
# Prediction: NOT ne_detected (True = predicted has evidence, False = predicted no evidence)

y_true = ~detailed_df['is_no_evidence_gt']  # True if has evidence
y_pred = ~detailed_df['ne_detected']  # True if predicted has evidence

# Confusion matrix components
tp = ((y_true) & (y_pred)).sum()  # True Positive: has evidence, predicted has evidence
tn = ((~y_true) & (~y_pred)).sum()  # True Negative: no evidence, predicted no evidence  
fp = ((~y_true) & (y_pred)).sum()  # False Positive: no evidence, predicted has evidence
fn = ((y_true) & (~y_pred)).sum()  # False Negative: has evidence, predicted no evidence

# Calculate all metrics
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0  # Sensitivity / TPR
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (tp + tn) / len(y_true)
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # True Negative Rate
npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value
balanced_accuracy = (recall + specificity) / 2
mcc_num = (tp * tn) - (fp * fn)
mcc_den = np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
mcc = mcc_num / mcc_den if mcc_den > 0 else 0  # Matthews Correlation Coefficient

print("=" * 80)
print("BINARY CLASSIFICATION METRICS: No-Evidence Detection (has_evidence vs no_evidence)")
print("=" * 80)

print("\n1. Confusion Matrix:")
print(f"                        Predicted")
print(f"                   Has Evidence | No Evidence")
print(f"   Actual Has Ev.       {tp:>5}    |    {fn:>5}      (Total: {tp+fn})")
print(f"   Actual No Ev.        {fp:>5}    |    {tn:>5}      (Total: {fp+tn})")
print(f"                   ---------------------")
print(f"   Total:               {tp+fp:>5}    |    {fn+tn:>5}      (Grand: {len(y_true)})")

print("\n2. Primary Metrics:")
print(f"   Accuracy:            {accuracy:.4f}  (Overall correctness)")
print(f"   Balanced Accuracy:   {balanced_accuracy:.4f}  (Accounts for class imbalance)")
print(f"   F1 Score:            {f1:.4f}  (Harmonic mean of precision & recall)")
print(f"   MCC:                 {mcc:.4f}  (Matthews Correlation Coefficient)")

print("\n3. Positive Class (Has Evidence) Metrics:")
print(f"   Precision (PPV):     {precision:.4f}  (Of predicted 'has evidence', how many correct)")
print(f"   Recall (Sensitivity):{recall:.4f}  (Of actual 'has evidence', how many detected)")

print("\n4. Negative Class (No Evidence) Metrics:")  
print(f"   Specificity (TNR):   {specificity:.4f}  (Of actual 'no evidence', how many detected)")
print(f"   NPV:                 {npv:.4f}  (Of predicted 'no evidence', how many correct)")

print("\n5. Class Distribution:")
n_has_evidence = y_true.sum()
n_no_evidence = (~y_true).sum()
print(f"   Has Evidence:        {n_has_evidence} ({n_has_evidence/len(y_true)*100:.1f}%)")
print(f"   No Evidence:         {n_no_evidence} ({n_no_evidence/len(y_true)*100:.1f}%)")
print(f"   Imbalance Ratio:     1:{n_no_evidence/n_has_evidence:.1f}")

BINARY CLASSIFICATION METRICS: No-Evidence Detection (has_evidence vs no_evidence)

1. Confusion Matrix:
                        Predicted
                   Has Evidence | No Evidence
   Actual Has Ev.         735    |      541      (Total: 1276)
   Actual No Ev.         2325    |     9909      (Total: 12234)
                   ---------------------
   Total:                3060    |    10450      (Grand: 13510)

2. Primary Metrics:
   Accuracy:            0.7879  (Overall correctness)
   Balanced Accuracy:   0.6930  (Accounts for class imbalance)
   F1 Score:            0.3390  (Harmonic mean of precision & recall)
   MCC:                 0.2697  (Matthews Correlation Coefficient)

3. Positive Class (Has Evidence) Metrics:
   Precision (PPV):     0.2402  (Of predicted 'has evidence', how many correct)
   Recall (Sensitivity):0.5760  (Of actual 'has evidence', how many detected)

4. Negative Class (No Evidence) Metrics:
   Specificity (TNR):   0.8100  (Of actual 'no evidence', how man

## 6.2 Multi-Label Metrics: Per-Post Accuracy Across All Criteria

For each post, multiple criteria need to be evaluated. This section measures how well we perform across all criteria for each post.

In [17]:
# Multi-Label Metrics: Per-Post Evaluation Across All Criteria
# Group by post_id to evaluate how well we do across all criteria for each post

# Add prediction correctness column
detailed_df['ne_correct'] = detailed_df['is_no_evidence_gt'] == detailed_df['ne_detected']

# Group by post_id
post_groups = detailed_df.groupby('post_id')

# Per-post metrics
post_metrics = []
for post_id, group in post_groups:
    n_criteria = len(group)
    n_correct = group['ne_correct'].sum()
    
    # Exact match: all criteria correct for this post
    exact_match = n_correct == n_criteria
    
    # Subset accuracy: fraction of criteria correct
    subset_acc = n_correct / n_criteria
    
    # Hamming score: 1 - hamming loss (same as subset accuracy for binary)
    hamming_score = subset_acc
    
    # Count evidence types
    n_has_evidence_gt = (~group['is_no_evidence_gt']).sum()
    n_no_evidence_gt = group['is_no_evidence_gt'].sum()
    n_has_evidence_pred = (~group['ne_detected']).sum()
    n_no_evidence_pred = group['ne_detected'].sum()
    
    post_metrics.append({
        'post_id': post_id,
        'n_criteria': n_criteria,
        'n_correct': n_correct,
        'exact_match': exact_match,
        'subset_accuracy': subset_acc,
        'hamming_score': hamming_score,
        'n_has_evidence_gt': n_has_evidence_gt,
        'n_no_evidence_gt': n_no_evidence_gt,
        'n_has_evidence_pred': n_has_evidence_pred,
        'n_no_evidence_pred': n_no_evidence_pred,
    })

post_metrics_df = pd.DataFrame(post_metrics)

print("=" * 80)
print("MULTI-LABEL METRICS: Per-Post Accuracy Across All Criteria")
print("=" * 80)

print(f"\n1. Dataset Overview:")
print(f"   Total posts:              {len(post_metrics_df)}")
print(f"   Total (post, criterion):  {len(detailed_df)}")
print(f"   Avg criteria per post:    {post_metrics_df['n_criteria'].mean():.2f}")
print(f"   Min/Max criteria:         {post_metrics_df['n_criteria'].min()} / {post_metrics_df['n_criteria'].max()}")

print(f"\n2. Exact Match Rate (All criteria correct for a post):")
exact_match_rate = post_metrics_df['exact_match'].mean()
n_exact_match = post_metrics_df['exact_match'].sum()
print(f"   Exact Match Rate:         {exact_match_rate:.4f} ({n_exact_match}/{len(post_metrics_df)} posts)")

print(f"\n3. Subset Accuracy (Avg fraction of criteria correct per post):")
subset_acc_mean = post_metrics_df['subset_accuracy'].mean()
subset_acc_std = post_metrics_df['subset_accuracy'].std()
print(f"   Mean Subset Accuracy:     {subset_acc_mean:.4f} +/- {subset_acc_std:.4f}")
print(f"   Median Subset Accuracy:   {post_metrics_df['subset_accuracy'].median():.4f}")

print(f"\n4. Hamming Score (1 - Hamming Loss):")
hamming_mean = post_metrics_df['hamming_score'].mean()
print(f"   Mean Hamming Score:       {hamming_mean:.4f}")

print(f"\n5. Per-Post Accuracy Distribution:")
acc_bins = [0, 0.25, 0.5, 0.75, 0.9, 1.0]
acc_labels = ['0-25%', '25-50%', '50-75%', '75-90%', '90-100%']
post_metrics_df['acc_bin'] = pd.cut(post_metrics_df['subset_accuracy'], bins=acc_bins, labels=acc_labels, include_lowest=True)
acc_dist = post_metrics_df['acc_bin'].value_counts().sort_index()
for label, count in acc_dist.items():
    print(f"   {label}: {count} posts ({count/len(post_metrics_df)*100:.1f}%)")

# Show worst and best performing posts
print(f"\n6. Post Performance Breakdown:")
perfect_posts = (post_metrics_df['subset_accuracy'] == 1.0).sum()
zero_posts = (post_metrics_df['subset_accuracy'] == 0.0).sum()
print(f"   Posts with 100% accuracy: {perfect_posts} ({perfect_posts/len(post_metrics_df)*100:.1f}%)")
print(f"   Posts with 0% accuracy:   {zero_posts} ({zero_posts/len(post_metrics_df)*100:.1f}%)")

MULTI-LABEL METRICS: Per-Post Accuracy Across All Criteria

1. Dataset Overview:
   Total posts:              1351
   Total (post, criterion):  13510
   Avg criteria per post:    10.00
   Min/Max criteria:         10 / 10

2. Exact Match Rate (All criteria correct for a post):
   Exact Match Rate:         0.1725 (233/1351 posts)

3. Subset Accuracy (Avg fraction of criteria correct per post):
   Mean Subset Accuracy:     0.7879 +/- 0.2053
   Median Subset Accuracy:   0.8000

4. Hamming Score (1 - Hamming Loss):
   Mean Hamming Score:       0.7879

5. Per-Post Accuracy Distribution:
   0-25%: 56 posts (4.1%)
   25-50%: 82 posts (6.1%)
   50-75%: 262 posts (19.4%)
   75-90%: 718 posts (53.1%)
   90-100%: 233 posts (17.2%)

6. Post Performance Breakdown:
   Posts with 100% accuracy: 233 (17.2%)
   Posts with 0% accuracy:   14 (1.0%)


## 6.3 Evidence Finding Accuracy (For Cases with Evidence)

For queries where the ground truth indicates evidence exists, how well do we find the correct evidence sentences?

In [18]:
# Evidence Finding Accuracy: For queries with actual evidence, how well do we find it?
# Filter to only queries that actually have evidence (ground truth)

has_evidence_results = [d for d in all_detailed_results if not d['is_no_evidence_gt']]

print("=" * 80)
print("EVIDENCE FINDING ACCURACY (For Queries with Ground Truth Evidence)")
print("=" * 80)

print(f"\n1. Dataset: {len(has_evidence_results)} queries with evidence")

# Compute evidence finding metrics
evidence_metrics = {k: {'hit_rate': [], 'precision': [], 'recall': [], 'ndcg': [], 'mrr': []} for k in [1, 3, 5, 10]}

all_map_scores = []
all_mrr_scores = []
n_perfect_recall = 0
n_zero_recall = 0

for result in has_evidence_results:
    gold_uids = set(result['gold_uids'])
    final_ids = result['final_ids']  # After post-processing
    reranked_ids = result['reranked_ids']  # Before post-processing
    
    if not gold_uids:
        continue
    
    # Calculate MRR (position of first relevant document)
    mrr = 0
    for i, uid in enumerate(reranked_ids):
        if uid in gold_uids:
            mrr = 1.0 / (i + 1)
            break
    all_mrr_scores.append(mrr)
    
    # Calculate MAP (Mean Average Precision)
    precisions = []
    num_relevant_found = 0
    for i, uid in enumerate(reranked_ids):
        if uid in gold_uids:
            num_relevant_found += 1
            precisions.append(num_relevant_found / (i + 1))
    ap = sum(precisions) / len(gold_uids) if gold_uids else 0
    all_map_scores.append(ap)
    
    # Metrics at different K values
    for k in [1, 3, 5, 10]:
        top_k_ids = set(final_ids[:k]) if final_ids else set()
        top_k_reranked = set(reranked_ids[:k])
        
        # Hit Rate: Did we find at least one relevant in top-k?
        hit = len(top_k_ids & gold_uids) > 0 if top_k_ids else 0
        evidence_metrics[k]['hit_rate'].append(hit)
        
        # Precision@K: What fraction of top-k are relevant?
        precision_k = len(top_k_ids & gold_uids) / k if top_k_ids else 0
        evidence_metrics[k]['precision'].append(precision_k)
        
        # Recall@K: What fraction of relevant did we find in top-k?
        recall_k = len(top_k_ids & gold_uids) / len(gold_uids) if gold_uids and top_k_ids else 0
        evidence_metrics[k]['recall'].append(recall_k)
        
        # nDCG@K (using reranked, before no-evidence filtering)
        ndcg_k = ndcg_at_k(gold_uids, reranked_ids, k)
        evidence_metrics[k]['ndcg'].append(ndcg_k)
    
    # Track perfect and zero recall
    if final_ids:
        final_recall = len(set(final_ids) & gold_uids) / len(gold_uids)
        if final_recall == 1.0:
            n_perfect_recall += 1
        elif final_recall == 0.0:
            n_zero_recall += 1

print(f"\n2. Global Ranking Metrics (using reranked results before filtering):")
print(f"   MRR (Mean Reciprocal Rank):  {np.mean(all_mrr_scores):.4f}")
print(f"   MAP (Mean Average Precision): {np.mean(all_map_scores):.4f}")

print(f"\n3. Metrics at Different K (using final post-processed results):")
print(f"   {'K':<5} {'Hit Rate':<12} {'Precision':<12} {'Recall':<12} {'nDCG':<12}")
print("   " + "-" * 53)
for k in [1, 3, 5, 10]:
    hr = np.mean(evidence_metrics[k]['hit_rate'])
    p = np.mean(evidence_metrics[k]['precision'])
    r = np.mean(evidence_metrics[k]['recall'])
    n = np.mean(evidence_metrics[k]['ndcg'])
    print(f"   {k:<5} {hr:<12.4f} {p:<12.4f} {r:<12.4f} {n:<12.4f}")

print(f"\n4. Evidence Coverage Analysis:")
print(f"   Queries with 100% recall:     {n_perfect_recall} ({n_perfect_recall/len(has_evidence_results)*100:.1f}%)")
print(f"   Queries with 0% recall:       {n_zero_recall} ({n_zero_recall/len(has_evidence_results)*100:.1f}%)")

# Distribution of gold evidence count
gold_counts = [len(d['gold_uids']) for d in has_evidence_results]
print(f"\n5. Gold Evidence Distribution:")
print(f"   Mean evidence per query:      {np.mean(gold_counts):.2f}")
print(f"   Median evidence per query:    {np.median(gold_counts):.1f}")
print(f"   Min/Max evidence:             {min(gold_counts)} / {max(gold_counts)}")

EVIDENCE FINDING ACCURACY (For Queries with Ground Truth Evidence)

1. Dataset: 1276 queries with evidence

2. Global Ranking Metrics (using reranked results before filtering):
   MRR (Mean Reciprocal Rank):  0.7062
   MAP (Mean Average Precision): 0.6969

3. Metrics at Different K (using final post-processed results):
   K     Hit Rate     Precision    Recall       nDCG        
   -----------------------------------------------------
   1     0.4208       0.4208       0.4097       0.5995      
   3     0.4757       0.1646       0.4710       0.6926      
   5     0.5086       0.1066       0.5049       0.7235      
   10    0.5086       0.0533       0.5049       0.7459      

4. Evidence Coverage Analysis:
   Queries with 100% recall:     639 (50.1%)
   Queries with 0% recall:       86 (6.7%)

5. Gold Evidence Distribution:
   Mean evidence per query:      1.13
   Median evidence per query:    1.0
   Min/Max evidence:             1 / 6


In [19]:
# Impact Analysis: How no-evidence detection errors affect evidence retrieval
# When we incorrectly predict "no evidence" for a query that has evidence (False Negative),
# we return empty results, causing 0% recall for that query.

print("=" * 80)
print("IMPACT ANALYSIS: No-Evidence Detection Errors on Evidence Retrieval")
print("=" * 80)

# Separate queries by detection correctness
has_evidence_queries = [d for d in all_detailed_results if not d['is_no_evidence_gt']]
correctly_detected = [d for d in has_evidence_queries if not d['ne_detected']]  # Correctly predicted has evidence
incorrectly_detected = [d for d in has_evidence_queries if d['ne_detected']]  # False negative (lost evidence)

print(f"\n1. Query Categories (Ground truth: has evidence):")
print(f"   Total queries with evidence:     {len(has_evidence_queries)}")
print(f"   Correctly detected (has_ev):     {len(correctly_detected)} ({len(correctly_detected)/len(has_evidence_queries)*100:.1f}%)")
print(f"   Incorrectly detected (no_ev):    {len(incorrectly_detected)} ({len(incorrectly_detected)/len(has_evidence_queries)*100:.1f}%)")
print(f"   → These {len(incorrectly_detected)} queries return EMPTY results (100% loss)")

# Compare metrics for correctly detected queries
if correctly_detected:
    correct_recalls = []
    correct_ndcgs = []
    for d in correctly_detected:
        gold = set(d['gold_uids'])
        final = set(d['final_ids'])
        if gold:
            recall = len(final & gold) / len(gold)
            correct_recalls.append(recall)
            ndcg = ndcg_at_k(gold, d['reranked_ids'], 10)
            correct_ndcgs.append(ndcg)
    
    print(f"\n2. Performance on Correctly Detected Queries:")
    print(f"   Mean Recall:   {np.mean(correct_recalls):.4f}")
    print(f"   Mean nDCG@10:  {np.mean(correct_ndcgs):.4f}")

# Calculate overall impact
if has_evidence_queries:
    all_recalls_with_ne = []
    all_recalls_without_ne = []
    for d in has_evidence_queries:
        gold = set(d['gold_uids'])
        if gold:
            # With no-evidence detection (using final_ids which is empty if ne_detected)
            final = set(d['final_ids'])
            recall_with = len(final & gold) / len(gold)
            all_recalls_with_ne.append(recall_with)
            
            # Without no-evidence detection (using all reranked results)
            reranked = set(d['reranked_ids'][:10])
            recall_without = len(reranked & gold) / len(gold)
            all_recalls_without_ne.append(recall_without)
    
    print(f"\n3. Overall Impact on Recall (queries with evidence):")
    print(f"   Recall WITH no-evidence detection:    {np.mean(all_recalls_with_ne):.4f}")
    print(f"   Recall WITHOUT no-evidence detection: {np.mean(all_recalls_without_ne):.4f}")
    print(f"   Impact (loss due to false negatives): {np.mean(all_recalls_without_ne) - np.mean(all_recalls_with_ne):+.4f}")

# Analyze what we lose in false negatives
if incorrectly_detected:
    lost_recalls = []
    for d in incorrectly_detected:
        gold = set(d['gold_uids'])
        reranked = set(d['reranked_ids'][:10])
        if gold:
            potential_recall = len(reranked & gold) / len(gold)
            lost_recalls.append(potential_recall)
    
    print(f"\n4. Lost Performance (False Negatives Analysis):")
    print(f"   Mean potential recall we lost:  {np.mean(lost_recalls):.4f}")
    print(f"   These queries had good evidence but we rejected them")

# Summary recommendation
print(f"\n5. Trade-off Summary:")
print(f"   - False Negatives (missed evidence): {len(incorrectly_detected)} queries ({len(incorrectly_detected)/len(has_evidence_queries)*100:.1f}%)")
no_evidence_queries = [d for d in all_detailed_results if d['is_no_evidence_gt']]
false_positives = [d for d in no_evidence_queries if not d['ne_detected']]
print(f"   - False Positives (hallucinated ev): {len(false_positives)} queries ({len(false_positives)/len(no_evidence_queries)*100:.1f}% of no-ev)")
print(f"   → FN causes complete loss of evidence for those queries")
print(f"   → FP returns results when there should be none")

IMPACT ANALYSIS: No-Evidence Detection Errors on Evidence Retrieval

1. Query Categories (Ground truth: has evidence):
   Total queries with evidence:     1276
   Correctly detected (has_ev):     735 (57.6%)
   Incorrectly detected (no_ev):    541 (42.4%)
   → These 541 queries return EMPTY results (100% loss)

2. Performance on Correctly Detected Queries:
   Mean Recall:   0.8765
   Mean nDCG@10:  0.8227

3. Overall Impact on Recall (queries with evidence):
   Recall WITH no-evidence detection:    0.5049
   Recall WITHOUT no-evidence detection: 0.8957
   Impact (loss due to false negatives): +0.3908

4. Lost Performance (False Negatives Analysis):
   Mean potential recall we lost:  0.8533
   These queries had good evidence but we rejected them

5. Trade-off Summary:
   - False Negatives (missed evidence): 541 queries (42.4%)
   - False Positives (hallucinated ev): 2325 queries (19.0% of no-ev)
   → FN causes complete loss of evidence for those queries
   → FP returns results when ther

## 6.4 Comprehensive Metrics Summary

A consolidated view of all evaluation metrics.

In [20]:
# Comprehensive Metrics Summary
print("=" * 90)
print("COMPREHENSIVE EVALUATION METRICS SUMMARY")
print("=" * 90)

print("\n" + "─" * 90)
print("PART A: BINARY CLASSIFICATION - No-Evidence Detection (has_evidence vs no_evidence)")
print("─" * 90)
print(f"""
┌────────────────────────────────────────────────────────────────────────────────────────┐
│  Metric                │  Value   │  Description                                       │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Accuracy              │  {accuracy:.4f}  │  Overall correctness                               │
│  Balanced Accuracy     │  {balanced_accuracy:.4f}  │  Accounts for class imbalance                      │
│  F1 Score              │  {f1:.4f}  │  Harmonic mean of precision & recall               │
│  MCC                   │  {mcc:.4f}  │  Matthews Correlation Coefficient                  │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Precision (PPV)       │  {precision:.4f}  │  Of predicted 'has evidence', how many correct     │
│  Recall (Sensitivity)  │  {recall:.4f}  │  Of actual 'has evidence', how many detected       │
│  Specificity (TNR)     │  {specificity:.4f}  │  Of actual 'no evidence', how many detected        │
│  NPV                   │  {npv:.4f}  │  Of predicted 'no evidence', how many correct      │
└────────────────────────┴──────────┴────────────────────────────────────────────────────┘
""")

print("\n" + "─" * 90)
print("PART B: MULTI-LABEL - Per-Post Accuracy Across All Criteria")
print("─" * 90)
print(f"""
┌────────────────────────────────────────────────────────────────────────────────────────┐
│  Metric                │  Value   │  Description                                       │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Exact Match Rate      │  {exact_match_rate:.4f}  │  Posts where ALL criteria predictions correct      │
│  Subset Accuracy       │  {subset_acc_mean:.4f}  │  Avg fraction of criteria correct per post         │
│  Hamming Score         │  {hamming_mean:.4f}  │  1 - Hamming Loss                                  │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Posts (100% correct)  │  {perfect_posts:>5}   │  {perfect_posts/len(post_metrics_df)*100:.1f}% of posts                                       │
│  Posts (0% correct)    │  {zero_posts:>5}   │  {zero_posts/len(post_metrics_df)*100:.1f}% of posts                                       │
└────────────────────────┴──────────┴────────────────────────────────────────────────────┘
""")

print("\n" + "─" * 90)
print("PART C: EVIDENCE FINDING - For Queries with Ground Truth Evidence")
print("─" * 90)
mrr_score = np.mean(all_mrr_scores)
map_score = np.mean(all_map_scores)
hr_at_1 = np.mean(evidence_metrics[1]['hit_rate'])
hr_at_5 = np.mean(evidence_metrics[5]['hit_rate'])
hr_at_10 = np.mean(evidence_metrics[10]['hit_rate'])
recall_at_5 = np.mean(evidence_metrics[5]['recall'])
recall_at_10 = np.mean(evidence_metrics[10]['recall'])
print(f"""
┌────────────────────────────────────────────────────────────────────────────────────────┐
│  Metric                │  Value   │  Description                                       │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  MRR                   │  {mrr_score:.4f}  │  Mean Reciprocal Rank (first relevant position)    │
│  MAP                   │  {map_score:.4f}  │  Mean Average Precision                            │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Hit Rate@1            │  {hr_at_1:.4f}  │  Found ≥1 relevant in top-1                        │
│  Hit Rate@5            │  {hr_at_5:.4f}  │  Found ≥1 relevant in top-5                        │
│  Hit Rate@10           │  {hr_at_10:.4f}  │  Found ≥1 relevant in top-10                       │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Recall@5              │  {recall_at_5:.4f}  │  Fraction of gold evidence found in top-5          │
│  Recall@10             │  {recall_at_10:.4f}  │  Fraction of gold evidence found in top-10         │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Perfect Recall        │  {n_perfect_recall:>5}   │  {n_perfect_recall/len(has_evidence_results)*100:.1f}% of queries found ALL evidence               │
│  Zero Recall           │  {n_zero_recall:>5}   │  {n_zero_recall/len(has_evidence_results)*100:.1f}% of queries found NO evidence                │
└────────────────────────┴──────────┴────────────────────────────────────────────────────┘
""")

# Store comprehensive metrics for saving
comprehensive_metrics = {
    # Binary classification
    'binary_accuracy': accuracy,
    'binary_balanced_accuracy': balanced_accuracy,
    'binary_f1': f1,
    'binary_mcc': mcc,
    'binary_precision': precision,
    'binary_recall': recall,
    'binary_specificity': specificity,
    'binary_npv': npv,
    # Multi-label per-post
    'multilabel_exact_match_rate': exact_match_rate,
    'multilabel_subset_accuracy': subset_acc_mean,
    'multilabel_hamming_score': hamming_mean,
    # Evidence finding
    'evidence_mrr': mrr_score,
    'evidence_map': map_score,
    'evidence_hit_rate_at_1': hr_at_1,
    'evidence_hit_rate_at_5': hr_at_5,
    'evidence_hit_rate_at_10': hr_at_10,
    'evidence_recall_at_5': recall_at_5,
    'evidence_recall_at_10': recall_at_10,
    'evidence_perfect_recall_rate': n_perfect_recall / len(has_evidence_results),
    'evidence_zero_recall_rate': n_zero_recall / len(has_evidence_results),
}

COMPREHENSIVE EVALUATION METRICS SUMMARY

──────────────────────────────────────────────────────────────────────────────────────────
PART A: BINARY CLASSIFICATION - No-Evidence Detection (has_evidence vs no_evidence)
──────────────────────────────────────────────────────────────────────────────────────────

┌────────────────────────────────────────────────────────────────────────────────────────┐
│  Metric                │  Value   │  Description                                       │
├────────────────────────┼──────────┼────────────────────────────────────────────────────┤
│  Accuracy              │  0.7879  │  Overall correctness                               │
│  Balanced Accuracy     │  0.6930  │  Accounts for class imbalance                      │
│  F1 Score              │  0.3390  │  Harmonic mean of precision & recall               │
│  MCC                   │  0.2697  │  Matthews Correlation Coefficient                  │
├────────────────────────┼──────────┼─────────────────

## 7. Save Results

In [21]:
# Save aggregated metrics
with open(OUTPUT_DIR / "aggregated_metrics.json", "w") as f:
    json.dump(aggregated, f, indent=2)

# Save comprehensive metrics (all three evaluation aspects)
with open(OUTPUT_DIR / "comprehensive_metrics.json", "w") as f:
    json.dump(comprehensive_metrics, f, indent=2)

# Save configuration
config = {
    'reranker_params': BEST_RERANKER_PARAMS,
    'postproc_params': BEST_POSTPROC_PARAMS,
    'retriever_config': config_metadata,
}
with open(OUTPUT_DIR / "config.json", "w") as f:
    json.dump(config, f, indent=2)

# Save per-fold metrics
fold_df.to_csv(OUTPUT_DIR / "fold_metrics.csv")

# Save per-post metrics
post_metrics_df.to_csv(OUTPUT_DIR / "post_metrics.csv", index=False)

# Save detailed results summary
detailed_df.to_csv(OUTPUT_DIR / "detailed_results.csv", index=False)

# Save full detailed results
with open(OUTPUT_DIR / "full_detailed_results.json", "w") as f:
    json.dump(all_detailed_results, f, indent=2)

print(f"Results saved to: {OUTPUT_DIR}")
print(f"  - aggregated_metrics.json     (ranking metrics by fold)")
print(f"  - comprehensive_metrics.json  (binary, multi-label, evidence finding)")
print(f"  - config.json                 (pipeline configuration)")
print(f"  - fold_metrics.csv            (per-fold breakdown)")
print(f"  - post_metrics.csv            (per-post multi-label metrics)")
print(f"  - detailed_results.csv        (per-query results)")
print(f"  - full_detailed_results.json  (complete detailed output)")

Results saved to: ../outputs/5fold_results_with_postproc
  - aggregated_metrics.json     (ranking metrics by fold)
  - comprehensive_metrics.json  (binary, multi-label, evidence finding)
  - config.json                 (pipeline configuration)
  - fold_metrics.csv            (per-fold breakdown)
  - post_metrics.csv            (per-post multi-label metrics)
  - detailed_results.csv        (per-query results)
  - full_detailed_results.json  (complete detailed output)


## 8. Final Summary

In [22]:
print("\n" + "="*90)
print("COMPLETE PIPELINE SUMMARY")
print("="*90)

print("\n1. Pipeline Configuration:")
print(f"   Retriever:       NV-Embed-v2 (top-{TOP_K_RETRIEVER})")
print(f"   Reranker:        {RERANKER_MODEL_ID.split('/')[-1]} (LoRA fine-tuned)")
ne_threshold = BEST_POSTPROC_PARAMS.get('score_gap_threshold', BEST_POSTPROC_PARAMS.get('score_std_threshold', 0))
print(f"   No-Evidence:     {BEST_POSTPROC_PARAMS['ne_method']} (threshold={ne_threshold:.4f})")
print(f"   Dynamic-K:       {BEST_POSTPROC_PARAMS['dk_method']} (k={BEST_POSTPROC_PARAMS['min_k']}-{BEST_POSTPROC_PARAMS['max_k']}, ratio={BEST_POSTPROC_PARAMS['score_gap_ratio']:.3f})")
print(f"   Calibration:     {BEST_POSTPROC_PARAMS['cal_method']}")

print("\n2. Performance (5-Fold CV, nDCG@10):")
print(f"   Retriever only:    {aggregated['ret_ndcg@10_mean']:.4f} +/- {aggregated['ret_ndcg@10_std']:.4f}")
print(f"   + Reranker:        {aggregated['rerank_ndcg@10_mean']:.4f} +/- {aggregated['rerank_ndcg@10_std']:.4f}")
print(f"   + Post-processing: {aggregated['final_ndcg@10_mean']:.4f} +/- {aggregated['final_ndcg@10_std']:.4f}")
print(f"   Dynamic-K nDCG:    {aggregated['dk_ndcg_mean']:.4f} +/- {aggregated['dk_ndcg_std']:.4f}")

print("\n3. Post-Processing Impact:")
print(f"   No-Evidence F1:    {aggregated['ne_f1_mean']:.4f} +/- {aggregated['ne_f1_std']:.4f}")
print(f"   Avg Dynamic K:     {aggregated['avg_dynamic_k_mean']:.2f} +/- {aggregated['avg_dynamic_k_std']:.2f}")

print("\n4. End-to-End Improvement:")
total_improvement = aggregated['rerank_ndcg@10_mean'] - aggregated['ret_ndcg@10_mean']
pct_improvement = total_improvement / aggregated['ret_ndcg@10_mean'] * 100
print(f"   Retriever -> Reranker: +{total_improvement:.4f} ({pct_improvement:+.1f}%)")

print("\n" + "="*90)


COMPLETE PIPELINE SUMMARY

1. Pipeline Configuration:
   Retriever:       NV-Embed-v2 (top-20)
   Reranker:        jina-reranker-v3 (LoRA fine-tuned)
   No-Evidence:     score_gap (threshold=1.1481)
   Dynamic-K:       score_gap (k=5-16, ratio=0.549)
   Calibration:     platt

2. Performance (5-Fold CV, nDCG@10):
   Retriever only:    0.6955 +/- 0.0193
   + Reranker:        0.7473 +/- 0.0233
   + Post-processing: 0.4639 +/- 0.0165
   Dynamic-K nDCG:    0.7251 +/- 0.0289

3. Post-Processing Impact:
   No-Evidence F1:    0.3432 +/- 0.0438
   Avg Dynamic K:     5.00 +/- 0.00

4. End-to-End Improvement:
   Retriever -> Reranker: +0.0518 (+7.4%)

